## Maximum likelihood mapmaking

In [ ]:
import maria
from maria.io import fetch

input_map = maria.map.load(fetch("maps/cluster2.fits"), nu=150e9)
input_map.data *= 5e1 

input_map[..., 256:-256, 256:-256].plot(cmap="cmb")
print(input_map)

In [ ]:
from maria import Planner

planner = Planner(target=input_map, site="cerro_toco", constraints={"el": (70, 90)})
plans = planner.generate_plans(total_duration=900, 
                               max_chunk_duration=900, 
                               sample_rate=50,
                               scan_options={"radius": input_map.width.deg / 2})

plans[0].plot()
print(plans)

In [ ]:
# import maria
from maria.instrument import Band

f150 = Band(
    center=150e9, 
    width=30e9, 
    NET_RJ=30e-6, 
    knee=1e1, 
    gain_error=2e-2)

array = {"field_of_view": 0.1, 
         "beam_spacing": 1.2,
         "primary_size": 50, 
         "shape": "circle",
         "bands": [f150]}

instrument = maria.get_instrument(array=array)

print(instrument)
instrument.plot()

In [ ]:
sim = maria.Simulation(
    instrument,
    plans=plans,
    site="cerro_toco",
    map=input_map,
    atmosphere="2d",
    atmosphere_kwargs={"weather": {"pwv": 1.0}},
)

print(sim)

In [ ]:
tods = sim.run()
tods[0].plot()

In [ ]:
from maria.mappers import MaximumLikelihoodMapper

ml_mapper = MaximumLikelihoodMapper(tods=tods, 
                                    width=0.8*input_map.width.deg,
                                    height=0.8*input_map.height.deg,
                                    units="Jy/pixel")
print(f"{ml_mapper.loss() = }")

The initial map is a "guess" constructing by heavily filtering the input TODs:

In [ ]:
print(ml_mapper.map)
ml_mapper.map.plot(cmap="cmb")


To fit the map we run

In [ ]:
ml_mapper.fit(epochs=4, steps_per_epoch=32, lr=1e-1)

which gives us an improved map

In [ ]:
ml_mapper.map.plot(cmap="cmb")

that will improve more as it continues to fit.